In [1]:
!pip install SpeechRecognition
! pip install PyDub

import os
import subprocess
import shlex
import pandas as pd
import numpy as np
import json
import os
import speech_recognition as sr

In [8]:
Symfuhny = {'url':'https://www.twitch.tv/videos/1797071998', 'name':'Symfuhny', 'beginning':'900', 'end':None, 'avg_viewers':5024, 'out_width':365, 'out_height':250, 'x':1298, 'y':241}
shroud = {'url':'https://www.twitch.tv/videos/1792790259', 'name':'shroud', 'beginning':'900', 'end':None, 'avg_viewers':11308, 'out_width':418, 'out_height':230, 'x':9, 'y':347}
cyr = {'url':'https://www.twitch.tv/videos/1797143072', 'name':'cyr', 'beginning':'900', 'end':None, 'avg_viewers':3437, 'out_width':396, 'out_height':367, 'x':1523, 'y':217}
NICKMERCS = {'url':'https://www.twitch.tv/videos/1798038394', 'name':'NICKMERCS', 'beginning':'3600', 'end':None, 'avg_viewers':14519, 'out_width':417, 'out_height':228, 'x':19, 'y':444}
Foolish_Gamers = {'url':'https://www.twitch.tv/videos/1798166416', 'name':'Foolish_Gamers', 'beginning':'900', 'end':None, 'avg_viewers':7856
, 'out_width':488, 'out_height':359, 'x':0, 'y':0}
Philza = {'url':'https://www.twitch.tv/videos/1798022118', 'name':'Philza', 'beginning':'900', 'end':None, 'avg_viewers':8163, 'out_width':554, 'out_height':330, 'x':0, 'y':398}
Xaryu = {'url':'https://www.twitch.tv/videos/1797143876', 'name':'Xaryu', 'beginning':'5400', 'end':None, 'avg_viewers':3473, 'out_width':518, 'out_height':373, 'x':11, 'y':561}
Ray = {'url':'https://www.twitch.tv/videos/1797237790', 'name':'Ray', 'beginning':'7200', 'end':None, 'avg_viewers':2134, 'out_width':433, 'out_height':203, 'x':12, 'y':71}
itsRyanHiga = {'url':'https://www.twitch.tv/videos/1799649695', 'name':'itsRyanHiga', 'beginning':'3600', 'end':None, 'avg_viewers':1656, 'out_width':357, 'out_height':259, 'x':0, 'y':92}
buckefps = {'url':'https://www.twitch.tv/videos/1797227298', 'name':'buckefps', 'beginning':'900', 'end':None, 'avg_viewers':2087, 'out_width':399, 'out_height':298, 'x':0, 'y':312}
Kastaclysm = {'url':'https://www.twitch.tv/videos/1798172429', 'name':'Kastaclysm', 'beginning':'3600', 'end':None, 'avg_viewers':1057, 'out_width':394, 'out_height':343, 'x':1525, 'y':0}

streamers = [Symfuhny, shroud, cyr, NICKMERCS, Foolish_Gamers, Philza, Xaryu, Ray, itsRyanHiga, buckefps, Kastaclysm]

In [9]:
# DIRECTORTY SETUP
os.chdir(r"C:\Users\trick\Desktop\twitchdownloader")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\trick\\Desktop\\twitchdownloader'

In [10]:
def FetchData(streamer):
    # DOWNLOAD CHAT LOGS
    command_line = f"TwitchDownloaderCLI.exe chatdownload --id {streamer['url']} -o {streamer['name']}-chat.json -b {streamer['beginning']}"
    if streamer['end'] is not None:
        command_line += f" -e {streamer['end']}"
    args = shlex.split(command_line)
    p = subprocess.Popen(args)
    p.wait()

    # GET DATA FROM CHAT LOGS
    data_json = json.load(open(f".../../../twitchdownloader/{streamer['name']}-chat.json", encoding="utf8"))
    streamer_name = data_json["streamer"]["name"]
    streamer_id = data_json["streamer"]["id"]
    end = (data_json["video"]["end"])
    end = end - ((end - int(streamer['beginning'])) % 15)
    comments = pd.DataFrame(data_json["comments"])
    data = comments["content_offset_seconds"].to_numpy()
    data = data[data < end]
    del comments

    # CALCULATE INTERACTIVITY
    interactivity = np.zeros(int((end - int(beginning)) / 15))
    for i in data:
        interactivity[(i - int(beginning)) // 15] += 1
    selected_indices = np.rint(np.arange(0,60) ** np.emath.logn(59,interactivity.shape[0]/2)).astype(int)
    interactivity_sorted = np.argsort(interactivity)[::-1][selected_indices]
    interactivity = interactivity / streamer['avg_viewers']

    # CREATE DATAFRAME, DOWNLOAD VIDEOS, CROP VIDEOS, AND EXTRACT AUDIO
    df = pd.DataFrame(columns=["streamer_name","streamer_id","start_seconds", "interactivity", "audio_path","video_path"])
    for i,j in enumerate(interactivity_sorted):
        command_line = f"TwitchDownloaderCLI.exe videodownload --id {streamer['url']} -o {streamer_name}-{i}.mp4 -b {j * 15 + int(streamer['beginning']) - 3} -e {(j+1) * 15 + int(streamer['beginning']) - 3}"
        args = shlex.split(command_line)
        p = subprocess.Popen(args)
        p.wait()
        command_line = f"ffmpeg -i {streamer_name}-{i}.mp4 -filter:v 'crop={streamer['out_width']}:{streamer['out_height']}:{streamer['x']}:{streamer['y']}' {streamer_name}-{i}-crop.mp4"
        args = shlex.split(command_line)
        p = subprocess.Popen(args)
        p.wait()
        command_line = f"ffmpeg -i {streamer_name}-{i}.mp4 {streamer_name}-{i}.mp3"
        args = shlex.split(command_line)
        p = subprocess.Popen(args)
        df = pd.concat([df,pd.DataFrame([{"streamer_name": streamer_name, "streamer_id": streamer_id, "start_seconds": j*15 + int(streamer['beginning']) - 3, "interactivity": interactivity[j],"audio_path":f"{streamer_name}-{i}.mp3","video_path":f"{streamer_name}-{i}.mp4"}])])
    df = df.reset_index().drop(columns=["index"])
    df.to_csv(f"{streamer_name}.csv")

In [11]:
# MOVE FILES TO OTHER FOLDER, MANUALLY DELETE VIDEOS THAT HAVE WRONG FACE CROPPING, THEN RUN THIS
def RenameFiles(streamer_name):
    os.chdir(f"C:\\Users\\trick\\Desktop\\twitchdownloader\\{streamer_name}")
    for i in range(60):
        if os.path.isfile(f"{streamer_name}-{i}-crop.mp4"):
            os.rename(f"{streamer_name}-{i}-crop.mp4", f"{streamer_name}-{i}.mp4")

In [12]:
for streamer in streamers:
    FetchData(streamer)

FileNotFoundError: [Errno 2] No such file or directory: 'TwitchDownloaderCLI.exe'

In [ ]:
for streamer in streamers:
    RenameFiles(streamer['name'])

In [31]:
# Concatenate data
df = pd.DataFrame(columns=["streamer_name","streamer_id","start_seconds", "interactivity", "audio_path","video_path"])
for streamer in streamers:
    temp_df = pd.read_csv(f"{streamer['name']}/{streamer['name']}.csv", index_col=0)
    temp_df['audio_path'] = temp_df['audio_path'].apply(lambda x: f"data/{streamer['name']}/{x}")
    temp_df['video_path'] = temp_df['video_path'].apply(lambda x: f"data/{streamer['name']}/{x}")
    df = pd.concat([df,temp_df])
df = df.reset_index().drop(columns=["index"])

In [26]:
# normalize interactivity
df['interactivity'] = df['interactivity'] / df['interactivity'].max()

In [30]:
df.to_csv('data.csv')

In [3]:
from pydub import AudioSegment
from pydub.exceptions import CouldntDecodeError

# Set the directory containing the audio files
directory = '/Users/meganshah/Desktop/Streamer-Interactivity-Analyzer/'

# List of directories to iterate over
dir_list = ['Xaryu', 'cyr', 'shroud', 'NICKMERCS', 'Kastaclysm', 'Symfuhny', 'Foolish_Gamers', 'Philza', 'itsRyanHiga', 'buckefps', 'Ray']

for dir_name in dir_list:
    dir_path = os.path.join(directory, dir_name)
    if os.path.isdir(dir_path):
        for filename in os.listdir(dir_path):
            if filename.endswith('.mp3'):
                file_path = os.path.join(dir_path, filename)
                wav_path = os.path.splitext(file_path)[0] + '.wav'

                # Convert mp3 to wav
                try:
                    sound = AudioSegment.from_file(file_path, format='mp3')
                    sound.export(wav_path, format='wav')
                except CouldntDecodeError:
                    print(f"Error: {file_path} could not be converted to wav format.")
                    continue

                # Transcribe the audio
                r = sr.Recognizer()
                with sr.AudioFile(wav_path) as source:
                    audio = r.record(source)

                try:
                    transcript = r.recognize_google(audio)
                except sr.UnknownValueError:
                    print(f"Error: Could not transcribe audio from {file_path}.")
                    continue

                # Write the transcript to a file
                transcript_dir = os.path.join(directory, 'transcripts', dir_name)
                os.makedirs(transcript_dir, exist_ok=True)
                transcript_filename = os.path.splitext(filename)[0] + '.txt'
                transcript_path = os.path.join(transcript_dir, transcript_filename)
                with open(transcript_path, 'w') as f:
                    f.write(transcript)
                


Xaryu-39.mp3: he's my brother's mind take this it is the first seed that fell from the life my remains nourished tell him my spirit remains here how the hell do I get there
Xaryu-11.mp3: popping off everyone saying yes I don't know even if retail was significantly better
Xaryu-10.mp3: or are you going to hire someone to come on the stream and cut it on stream not that's the other option but who do I really trust to do that you know what I'm saying
Xaryu-38.mp3: I only have a couple spots left
Xaryu-12.mp3: play some different music I'm listening to that stranglethorn Vale soundtrack and you want me to switch them
Xaryu-13.mp3: cast for me it's agreed from me on real well we got five Quest complete in this dungeon are those emojis away seed of life
Error: Could not transcribe audio from /Users/meganshah/Desktop/Streamer-Interactivity-Analyzer/Xaryu/Xaryu-17.mp3.
Xaryu-16.mp3: like some type of a complete idiot I don't know I don't know
Xaryu-14.mp3: a zombies and yeah I hope you guys ha

cyr-18.mp3: did everyone that was working in this area the country back then had to communicate over long distances that used hollering as a form of May today unication I need to go to spivey's Corner Instagram
cyr-24.mp3: being 20 years behind everybody then I would be affecting a lot of places and being so miserable you don't like your neighbor you don't speak your neighbor you're you're better-looking World Atlanta's good example
cyr-30.mp3: I don't even hear it I already know the context of this is a show I doesn't bother me cuz I don't have mastered ignoring laugh tracks
cyr-5.mp3: doesn't stop sold and despite living in the twenty-first century I love living in the past 12
cyr-1.mp3: this part is this part is acting stop Mountain Wendy
cyr-34.mp3: like in Harpers I got back it was just loud to me let's get back to this and Landing really predictably where were you wanted to
cyr-20.mp3: I don't think that's why I don't think he's going to be our Mountain representative representat

shroud-25.mp3: yeah
shroud-24.mp3: alright
shroud-30.mp3: the division map out there now
Error: Could not transcribe audio from /Users/meganshah/Desktop/Streamer-Interactivity-Analyzer/shroud/shroud-18.mp3.
shroud-34.mp3: play this play this game on my my phone is playing right now
shroud-20.mp3: the person with the little bottle HP bottle okay well
Error: Could not transcribe audio from /Users/meganshah/Desktop/Streamer-Interactivity-Analyzer/shroud/shroud-21.mp3.
shroud-35.mp3: drop your damn clothes I kind of can't I can't put your heads down give me a shoes
shroud-23.mp3: good cak seems kind of hot ass I'll be honest with you guys but maybe you just need to get all of the fucken attachments that I don't know
shroud-37.mp3: what is a female troll
shroud-36.mp3: play the the the real game like we usually when you get a beta it's it's the actual game if a beta truly is a beta where you know for a fact that it's going to be
shroud-22.mp3: Limonite
shroud-13.mp3: traffic 105 never seen 

NICKMERCS-36.mp3: also next time to do a cookout
NICKMERCS-20.mp3: making you might finally have a scary scary movie Banger hook up the trailer talk to me
NICKMERCS-34.mp3: nice might be a reach I am a nice guy especially now bro
NICKMERCS-35.mp3: what's the radar as when we were like marrying them and we just didn't let it get a little crazy and then walk in after we would definitely all then
NICKMERCS-21.mp3: King Kong
Kastaclysm-49.mp3: wake Chloe 415 $60 to keep an eye out as often on sale I remember days gone was promoted at one of the pax's I think that's why I
Kastaclysm-48.mp3: okay back here though there should be something by the stone
Kastaclysm-8.mp3: Lansdowne most competitive sport
Kastaclysm-9.mp3: it is what it is
Kastaclysm-58.mp3: I'm thinking 20 different things at the same time so I can pray over charge to flip this the murder over charge to put that one price
Kastaclysm-59.mp3: greasy greasy money money money money I kind of like it giving me more what is the wine 

Symfuhny-17.mp3: garnjana Vegas
Symfuhny-16.mp3: tagawa probably like hopefully in like a month we have ranks hopefully hopefully hopefully
Symfuhny-14.mp3: prancercise
Symfuhny-28.mp3: that Tisha
Symfuhny-29.mp3: black black who can we get for a third.
Symfuhny-15.mp3: 5 add back-to-backs is crazy there enough for
Symfuhny-59.mp3: nothing they haven't got off yet
Symfuhny-58.mp3: finally thank goodness now
Symfuhny-48.mp3: Yelp
Symfuhny-49.mp3: yeah
Symfuhny-9.mp3: if my card goes there I get the bill to yourself but I don't know if it will even get the client
Symfuhny-8.mp3: it was Randy in the first nonstop was Jam Jeremy Lin been falling since 2020 we like that
Symfuhny-5.mp3: watching controller players trying to shoot through smoke
Symfuhny-53.mp3: I'm going to follow them up news
Symfuhny-47.mp3: that they didn't want to play with me and that's why they did that sound like
Symfuhny-46.mp3: isn't 3.5 deals on the.
Symfuhny-52.mp3: Akinator
Symfuhny-4.mp3: don't worry. That happen

Foolish_Gamers-48.mp3: because I saw recover all your hearts Mis corazones I know Hearts corazones favorite kind of looks like recover a camera that I think all your soy that I
Foolish_Gamers-49.mp3: I'll do like that.
Foolish_Gamers-8.mp3: play jazz robots about tomorrow at some undesignated undisclosed time see you when I see you
Foolish_Gamers-9.mp3: let's see, you are not ready to say you are not
Philza-19.mp3: it's a poppy flower a rose that my favorite
Philza-25.mp3: night and it's two-sided
Philza-31.mp3: peaches try to keep my paintings should I try to get along with you I feel like you should always err on the side of caution with sketchy people alright
Philza-30.mp3: wake up wake up your dad about going to get on for a little bit he's going to be late wake up long enough to do the quest Quest
Philza-24.mp3: people playing the doll until you're ready come out good morning good morning it's time to do quests he who fortunately the quest. 22 dangerous
Philza-18.mp3: we like the 

itsRyanHiga-23.mp3: going on here do you hear sound again what happened I had no sound
itsRyanHiga-6.mp3: there's no way I find to shiny Shinx has and the thing is not
itsRyanHiga-7.mp3: what color is the Shiny Beldum it's silver
itsRyanHiga-22.mp3: and also going for a catch combos so every under everyone's
itsRyanHiga-36.mp3: I guess back to the Grind will actually do anything berries I should we have way more than enough
itsRyanHiga-51.mp3: 2
itsRyanHiga-45.mp3: night thunderstorms even better yep I would ruin my catch combo for that
itsRyanHiga-44.mp3: do other people know that about catch combos
itsRyanHiga-50.mp3: come on shiny 116 chance
itsRyanHiga-46.mp3: another Clefairy
itsRyanHiga-52.mp3: we can get all this this week struggle bug
Error: Could not transcribe audio from /Users/meganshah/Desktop/Streamer-Interactivity-Analyzer/itsRyanHiga/itsRyanHiga-53.mp3.
Error: Could not transcribe audio from /Users/meganshah/Desktop/Streamer-Interactivity-Analyzer/itsRyanHiga/itsRyanHiga

Ray-29.mp3: if I Emily who else do I have to know that worked out
Ray-15.mp3: got a lot of work to do for someone to overtake teen on the spit War but we'll see we'll see thank everybody thank you will be back tomorrow for shorter stream we're going to check out the new Minecraft Legends game that came out today
Ray-14.mp3: what's up I just gave him my gold and parsnip
Ray-28.mp3: you work part-time at jojamart you fucking capitalist if they were Jack Black and he would kill it
Ray-16.mp3: we're closed this week and I'm like fire-sale fire-sale and I'm also we talked about not necessarily like
Ray-17.mp3: you think you know a guy you think you know a guy its you know it doesn't matter you tried to cheat all right
Ray-13.mp3: a butterfish going to stick a butter but it's a fish nice
Ray-12.mp3: Anythink stream was not starting great but I'm being so brave cuz I'd only were there technical issues but I got a tummy ache all day we went to get sushi Essay with a bunch of friends with two f